### 벡터 DB 생성 및 저장
- 식물 데이터 벡터 DB 인덱스 생성 (PineCone DB 사용)
- 데이터셋 임베딩 (gpt 모델)
- 데이터셋 저장

In [43]:
import json
import pandas as pd

# 데이터 로드
with open('./../../datas/flower_preprocessed_data.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# 데이터 전처리
def _create_text_for_embedding(row):
    air_cond_text = '공기정화식물이다.' if row['isAirCond'] else ''
    toxic_dog_text = '강아지에게 위험하다.' if row['isToxicToDog'] else '강아지에게 안전하다.'
    toxic_cat_text = '고양이에게 위험하다.' if row['isToxicToCat'] else '고양이에게 안전하다.'
    colors = " ".join(row['colors'])

    return (
        f"{row['flowLang']}, "
        f"{row['fContent']}, "
        f"물주기: {row['watering_frequency']}, "
        f"난이도 {row['difficulty']}, "
        f"{row['interior']}, "
        f"{row['style']}, "
        f"{row['fUse']}, "
        f"{row['fGrow']}, "
        f"{row['fType']}. "
        f"{air_cond_text} "
        f"{toxic_dog_text} "
        f"{toxic_cat_text} "
        f"{colors} "
    ).strip()

df = pd.DataFrame(data)
df['text'] = df.apply(_create_text_for_embedding, axis=1)
texts = df['text'].tolist()

In [44]:
df.columns

Index(['dataNo', 'fMonth', 'fDay', 'flowNm', 'fSctNm', 'fEngNm', 'flowLang',
       'fContent', 'fUse', 'fGrow', 'fType', 'fileName1', 'fileName2',
       'fileName3', 'imgUrl1', 'imgUrl2', 'imgUrl3', 'publishOrg', 'isAirCond',
       'isToxicToDog', 'isToxicToCat', 'colors', 'watering_frequency',
       'difficulty', 'interior', 'style', 'text'],
      dtype='object')

In [45]:
df.iloc[0]['text']

'영원한 향기, 향나무는 상록침엽수로 우리나라에서 가장 오래 사는 나무 중의 하나이다. 예로부터 마음을 담아 기원을 하는 나무로 소중히 여겨왔으며 관상, 약용, 향료, 조각재 등으로 널리 이용되어 왔다. 목질도 단단하려니와 목재의 향과 색이 일품인 이유로 보인다. 울릉도 도동에 사는 향나무는 2,500년생으로 세계에서 가장 오래된 것으로 전해지고 있다., 물주기: 주 1회, 난이도 쉬움, 향나무는 잔잔한 향기를 지니고 있어 침실이나 거실에 놓아 두면 향기로움과 함께 편안한 분위기를 조성할 수 있습니다. 화분에 심어 책상이나 선반 위에 두거나, 인테리어 선반에 장식으로 활용하면 좋습니다., 스칸디나비아, 미니멀리즘, 보헤미안, 향나무는 입지조건을 크게 구애받지 않아 건물주변의 녹지조성이나 가로수로 이용하고 있으며 가지가 곧게 또는 구부러지는 모양새를 하고 있어 정원수나 생울타리로 애용되고 있고 분재용으로도 쓰이고 있다. 특히 공해와 추위에 강해 전국에서 월동할 수 있어 생활주변에서 가장 널리 쓰이는 상록교목 중의 하나이다., 어릴 때 성장은 더디며 뾰족한 침엽을 보이지만 10년 이상 지나면 침엽이 비늘잎형태로 변하면서 성장도 빨라진다. 열매는 땅에 떨어져 스스로 발아되는 경우는 드물고 새 먹이가 되어 배설물에 섞여 나오면 과육에 있는 발아억제물질이 제거되어 싹이 나게 된다. 대부분은 꺾꽂이로 증식하는데, 봄에 본격적으로 나오는 새순을 잘라 꽂으면 뿌리가 잘 내린다., 측백나무과 향나무속에 속하며 전국에 자생 또는 식재되어 있다. 종류는 자생하거나 흔히 심겨져 있는 향나무, 줄기가 누운 형태로 자라는 눈향나무, 고산지대에서만 자라는 곱향나무, 가지가 수평으로 퍼지는 뚝향나무, 북아메리카원산인 연필향나무 그리고 가지가 나선모양으로 돌아가는 가이쓰가향나무가 있으며 주로 이용되는 것은 가이쓰가향나무와 연필향나무다..  강아지에게 안전하다. 고양이에게 안전하다. #363B20 #9D9E6D #181F10 #96AE6F #33331C #959878'

In [47]:
# Pinecone 객체 생성
from pinecone import Pinecone, ServerlessSpec
from dotenv import load_dotenv
import os

load_dotenv()
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

pc = Pinecone(api_key=PINECONE_API_KEY)

In [48]:
# pc 관련 함수
pc.delete_index('plant-recommend')   # 인덱스 삭제

In [49]:
pc.list_indexes()            # 인덱스 리스트 확인

[
    {
        "name": "plant-qna",
        "metric": "cosine",
        "host": "plant-qna-ve21van.svc.aped-4627-b74a.pinecone.io",
        "spec": {
            "serverless": {
                "cloud": "aws",
                "region": "us-east-1"
            }
        },
        "status": {
            "ready": true,
            "state": "Ready"
        },
        "vector_type": "dense",
        "dimension": 1536,
        "deletion_protection": "disabled",
        "tags": null
    }
]

In [50]:
# 인덱스 생성
index_name = 'plant-recommend'

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric='cosine',
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )

In [51]:
# 벡터 DB에 저장
from langchain_openai import OpenAIEmbeddings

# 임베딩 모델 생성
embedding_model = OpenAIEmbeddings(model='text-embedding-3-small')
embeddings = embedding_model.embed_documents(texts)

# 인덱스 불러오기
rec_index = pc.Index(index_name)

for i, row in df.iterrows():
    metadatas = row.drop(['dataNo', 'fSctNm', 'fEngNm', 'fileName1', 'fileName2', 'fileName3', 'publishOrg', 'colors']).to_dict()

    # 인덱스에 upsert
    rec_index.upsert([
        (
            str(row['dataNo']),
            embeddings[i],
            metadatas
        )
    ])

### RAG & LLM 모델 연결
- retriever 생성 후 쿼리로 질의
- RAG로 유사한 데이터 찾은 후 LLM 모델에 전달
- LLM 모델: GPT-4o
- mockdata로 테스트

In [39]:
# mockdata
messages = [
  {
    "role": "assistant",
    "content": "안녕하세요! 공간에 어울리는 식물을 추천해드릴게요. 우선 어떤 공간에 두실 예정인가요?"
  },
  {
    "role": "user",
    "content": "우리 집 거실에 둘 식물 찾고 있어."
  },
  {
    "role": "assistant",
    "content": "좋아요! 그럼 거실은 햇빛이 얼마나 들어오는 편인가요? 강하게 들어오나요, 은근하게 들어오나요, 아니면 거의 안 들어오나요?"
  },
  {
    "role": "user",
    "content": "은근하게 들어오는 정도야."
  },
  {
    "role": "assistant",
    "content": "은은한 채광이면 반음지 식물도 잘 맞을 것 같아요. 혹시 선호하는 인테리어 스타일 있으신가요? 예: 미니멀, 내추럴, 화려함 등"
  },
  {
    "role": "user",
    "content": "나는 미니멀한 스타일 좋아해."
  },
  {
    "role": "assistant",
    "content": "좋습니다! 그럼 색감은 어떤 톤을 원하시나요? 올리브톤, 짙은 초록, 파스텔톤 등 좋아하는 색 있으신가요?"
  },
  {
    "role": "user",
    "content": "올리브톤이랑 자연스러운 초록 좋아해."
  },
  {
    "role": "assistant",
    "content": "취향이 딱 잡혀 있네요! 마지막으로 관리 난이도는 어떤 걸 원하세요? 초보자용, 어느 정도 신경 써야 하는 식물, 상관 없음?"
  },
  {
    "role": "user",
    "content": "초보자용으로 쉬운 게 좋아."
  },
  {
    "role": "assistant",
    "content": "완벽해요! 지금까지 정보면 충분합니다. 알려주신 거실 환경과 취향, 선호 스타일에 맞춰 몇 가지 식물 추천 리스트 만들어드릴까요?"
  },
  {
    "role": "user",
    "content": "응 만들어줘!"
  }
]

collected_data = {
    "room": "거실 / 거실의 빈 벽이 너무 심심해서 놓고 싶어",
    "humidity": "건조",
    "preferred_style": "미니멀",
    "preferred_color": "따뜻한 색",
    "has_dog": True,
    "has_cat": False,
    "isAirCond": True,
    "watering_frequency": "주 1회",
    "user_experience": "쉬움"
}

prev_results = ['튤립', '사랑초']

In [40]:
# RAG
from langchain_pinecone import PineconeVectorStore

# 인덱스 불러오기
vector_store = PineconeVectorStore(
    index_name=index_name,
    embedding=embedding_model
)

# 검색
retriever = vector_store.as_retriever(search_kwargs={'k': 1})
query = str(collected_data)
retrievals = retriever.batch([query])

In [42]:
retrievals

[[Document(id='6', metadata={'difficulty': '쉬움', 'fContent': '공기정화식물도 잘 알려져 있다. 손가락같이 갈라진 잎과 뿌리 윗부분의 줄기가 통통한 모양새가 특이한 이국적인 식물이다. 자생지인 멕시코에서는 교목성으로 자라지만 우리나라에서는 화분에 심어 잎을 관상하는 관엽식물로 자리잡고 있다.', 'fDay': '6', 'fGrow': '전형적인 남방계식물로 추위에 약해 겨울에도 최저 10℃ 이상은 되어야 별탈없이 월동할 수 있다. 보통은 16-30℃정도가 알맞다. 실내라 하더라도 햇볕이 잘 드는 곳이 좋으며 환기만 잘 된다면 습도가 높을수록 잘 자란다.', 'fMonth': '1', 'fType': '물밤나무과에 속하며 원산지인 남미에서는 열매를 식용으로 쓰기도 한다.', 'fUse': '잎은 시원하고 줄기는 매끈하며 특히 줄기 아랫부분은 항아리모양으로 불룩하게 비대되어 모양이 특이하며 보기가 좋아 집안의 거실이나 건물의 실내식물로 잘 어울린다. 현재 이용되는 것은 대형종이 많지만 키가 작은 소형종도 있다.', 'flowLang': '행운', 'flowNm': '파키라', 'imgUrl1': 'https://www.nihhs.go.kr/Attfiles/FLOWF/0106-3.jpg', 'imgUrl2': 'https://www.nihhs.go.kr/Attfiles/FLOWF/0106-2.jpg', 'imgUrl3': 'https://www.nihhs.go.kr/Attfiles/FLOWF/0106-1.jpg', 'interior': '파키라는 실내에서 높고 넓은 공간을 채우기 좋으며, 화이트나 연한 색상의 화분에 심어 포인트를 줄 수 있습니다. 또한, 잎이 풍성해 공간에 생동감을 더해줍니다.', 'isAirCond': False, 'isToxicToCat': False, 'isToxicToDog': True, 'style': '모던, 스칸디나비안, 보헤미안', 'watering_frequency': '주 1회'}, 

In [ ]:
# LLM
from langchain_core.messages import SystemMessage
from langchain_openai import ChatOpenAI

prompt= f"""
    ### 요구사항 ###
    {collected_data}

    ### 이미 추천한 식물 (다시 추천하지 않습니다) ###
    {prev_results}

    ### 설명 ###
    당신은 친절한 식물 전문가입니다. 한국어로 친절하게 답변하세요.
    반드시 JSON만 출력하세요. JSON 앞뒤에 설명, 문장, 코드블록, ``` 표시를 넣지 마세요.
    RAG 검색 결과를 참고해서 사용자에게 식물 1가지를 추천하세요.
    이미 추천한 식물은 사용자가 거부한 식물이니 추천하지 않습니다.

    ### 응답목록 ###
    - 식물 이름
    - 추천하는 이유 & 식물의 특징

    ### 출력형식 ###
    {{
        "flowNm": "식물 이름",
        "response": "추천하는 이유 & 식물의 특징"
    }}
"""

system_msg = SystemMessage(prompt)
input_msg = [system_msg] + messages

model = ChatOpenAI(
    model = 'gpt-4o',
    temperature=1
)

recommend_result = ''
response = model.invoke(input_msg)

while True:
    response = model.invoke(input_msg)

    try:
        res_json = json.loads(response.content)
        recommend_result = res_json['flowNm']
        break
    except:
        continue